In [3]:
%pylab inline
%load_ext autoreload
%autoreload 2
import numpy as np
import numpy
import matplotlib.pyplot as plt
import pandas as pd
import sys
import os
import six
sys.path.append("/Users/uchihashi_k/Desktop/projects/lb_time/utils")

import sklearn.metrics
from sklearn.metrics import precision_recall_curve
from sklearn.metrics import auc
from sklearn.metrics import roc_curve
from sklearn.grid_search import GridSearchCV
import sklearn
from sklearn.externals import joblib
from scipy.spatial.distance import euclidean

from fastdtw import fastdtw

Populating the interactive namespace from numpy and matplotlib
The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


`%matplotlib` prevents importing * from pylab and numpy


In [4]:
import seaborn as sns

import function
import preprocessing
import local_bayes
import visualize
import metrics

In [5]:
sys.path.append("/Users/uchihashi_k/Desktop/projects/lb_time/models/cnn")
import cnn

# nearest points

In [13]:
save_data_dir = '../data/bookmark/'
train_0_0 = numpy.load(save_data_dir + '0_test/0_fold/0_train.npz')
test_0_0 = numpy.load(save_data_dir + '0_test/0_fold/0_test.npz')

In [14]:
new_tr = train_0_0['x'][:1000]
new_te = test_0_0['x'][0]

In [132]:
localbayes = local_bayes.LocalBayes(nn_num=20, data_dim=30)

In [16]:
nn_dist_ls, nn_ts_ls = localbayes.get_nearest_n(train_0_0['x'], new_te)

758

In [135]:
len(nn_ts_ls)

20

In [18]:
numpy.array(nn_ts_ls)

array([[ 2,  1,  1,  0,  7,  3,  4,  7, 11,  6,  4,  4,  7,  5,  5,  7,  1,
         3,  5,  5,  7,  0,  1,  3,  2,  2,  4,  3,  1,  4],
       [ 2,  1,  2,  8,  1,  2,  2,  8,  5,  1,  4,  4,  8, 10,  2,  2,  0,
         2,  2,  2,  2,  1,  2,  2,  1,  0,  2,  2,  6,  0],
       [ 3,  2,  3,  5,  7,  1,  7,  1,  1,  1,  1,  2,  4,  1,  1,  1,  0,
         1,  2,  4,  3,  1,  3,  1,  2,  3,  0,  4,  0,  1],
       [ 0,  2,  1,  6, 16, 13,  3,  5,  6,  1,  2,  1,  5,  0,  1,  3,  7,
         1,  1,  1,  1,  1,  1,  1,  2,  1,  0,  3,  0,  1],
       [ 1,  9,  6,  2, 14, 10,  5,  5,  1,  2,  4,  1,  2,  1,  6,  0,  4,
         5,  3,  2,  0,  0,  3,  4,  4,  3,  2,  0,  2,  1],
       [ 0,  2,  5,  2,  6,  4,  5,  1,  4,  3,  2,  1,  4,  1,  0,  1,  3,
         5,  2,  1,  3,  2,  2,  1,  2,  1,  1,  1,  1,  1],
       [ 0,  0,  0,  0,  2,  5, 11,  4,  1,  0,  2,  0,  2,  2,  2,  3,  1,
         1,  1,  1,  1,  6,  0,  3,  2,  3,  1,  3,  4,  0],
       [ 0,  0,  0,  0,  7, 13, 22,  8,  

## prediction

### gradient boosting decision tree

In [19]:
from sklearn.ensemble import AdaBoostClassifier
from sklearn.tree import DecisionTreeClassifier

In [20]:
max_depth = 4
n_estimators = 5000

In [21]:
bdt = joblib.load('../models/gbdt/results/bdt_%s_%s.pkl' % (max_depth, n_estimators))

In [54]:
gbdt_pred = bdt.predict_proba(numpy.array(nn_ts_ls))
gbdt_pred = gbdt_pred[:, 1].tolist()
gbdt_pred

[0.49595700562528316,
 0.4963279326980058,
 0.5018216740450403,
 0.5001240860643253,
 0.5020297148593013,
 0.5006630041875523,
 0.49494918737213006,
 0.48509078479105927,
 0.49769480945527955,
 0.49882025072187747,
 0.49312866101663677,
 0.4945560017422757,
 0.5019556691731124,
 0.5015629959725285,
 0.5009917399909998,
 0.49657133035668094,
 0.49769480945527955,
 0.49734666478262257,
 0.4833935677247825,
 0.4968898831320494]

### CNN

In [23]:
import chainer
import chainer.functions as F
import chainer.links as L
from chainer import cuda
from chainer import optimizers
from chainer import serializers
from chainer import Link, Chain, ChainList

import pandas as pd
from ggplot import *

In [24]:
save_model_dir = '../models/cnn/results/awesome_100/'

In [29]:
sys.path.append("/Users/uchihashi_k/Desktop/projects/lb_time/models/cnn")
import cnn_net

In [36]:
N_test = numpy.array(nn_ts_ls).shape[0]
data_dim = test_0_0['x'].shape[1]

In [32]:
model = L.Classifier(cnn_net.tsCNN_net(data_dim))

In [33]:
serializers.load_npz(save_model_dir + "mlp.model", model)

In [34]:
def forward(x_data, model):
    x = chainer.Variable(x_data)
    h = F.max_pooling_2d(F.relu(model.conv1(x)), 2)
    h = F.max_pooling_2d(F.relu(model.conv2(h)), 2)
    h = F.relu(model.l1(h))
    y = model.l2(h)

    return y

In [40]:
nn_x = numpy.array(nn_ts_ls).reshape((N_test, 1, 1, data_dim))

In [44]:
cnn_pred = []
for i in range(0, N_test):
    x = np.asarray(nn_x[i]).reshape((1, 1, 1, data_dim)).astype(numpy.float32)
    
    pred_ = F.softmax(model.predictor(chainer.Variable(x)))
    cnn_pred.append(pred_.data[0][1])

In [46]:
cnn_pred

[0.0026402085,
 9.3086167e-10,
 0.99981683,
 0.99943095,
 0.99990177,
 0.98906881,
 5.6098847e-06,
 1.2144666e-16,
 0.00092398445,
 0.0015024451,
 0.0041209068,
 1.322026e-07,
 0.99985313,
 0.99979764,
 0.99630016,
 0.0026082713,
 0.00092398445,
 0.024740428,
 2.0070509e-08,
 0.0031541879]

## local bayes estimation

In [99]:
learner_name_ls = ['cnn', 'gbdt']
pred_proba_ls = [cnn_pred, gbdt_pred]

In [106]:
w_dict = localbayes.local_bayes_estimation(pred_proba_ls, learner_name_ls)

In [107]:
w_dict

{'cnn': 0.0, 'gbdt': 1.0}

In [130]:
model_dict = {}
model_dict.update({'cnn':model})
model_dict.update({'gbdt':bdt})

In [134]:
lb_pred = localbayes.local_bayes_predict(new_te, learner_name_ls, w_dict, model_dict)

/Users/uchihashi_k/anaconda/lib/python2.7/site-packages/sklearn/utils/validation.py:386: DeprecationWarning: Passing 1d arrays as data is deprecated in 0.17 and willraise ValueError in 0.19. Reshape your data either using X.reshape(-1, 1) if your data has a single feature or X.reshape(1, -1) if it contains a single sample.
  DeprecationWarning)


In [47]:
import pulp

In [59]:
learner_name_ls = ['cnn', 'gbdt']
pred_proba_ls = [cnn_pred, gbdt_pred]

In [86]:
problem = pulp.LpProblem('localbayes', pulp.LpMaximize) # 最大化する場合

In [87]:
var = pulp.LpVariable.dicts('w', (learner_name), 0, 1, 'Continuous')

In [88]:
var

{'cnn': w_cnn, 'gbdt': w_gbdt}

In [89]:
## objective
for j in six.moves.range(20):
    model_sum = 0
    
    for i, learner in enumerate(learner_name):
        model_sum += var[learner] * math.log(pred_proba_ls[i][j] + 0.000000001)
        
    problem += model_sum

##  constraint
problem_ = 0

for learner in learner_name:
    problem += var[learner] >= 0
    problem_ += var[learner]
    
problem += problem_ == 1

In [90]:
model_sum

LpAffineExpression([(w_cnn, -5.759023890876643),
                    (w_gbdt, -0.6993868385363117)])

In [91]:
problem

localbayes:
MAXIMIZE
-5.75902389088*w_cnn + -0.699386838536*w_gbdt + -0.0
SUBJECT TO
_C1: w_cnn >= 0

_C2: w_gbdt >= 0

_C3: w_cnn + w_gbdt = 1

VARIABLES
w_cnn <= 1 Continuous
w_gbdt <= 1 Continuous

In [92]:
status = problem.solve()

In [93]:
pulp.LpStatus[status]

'Optimal'

In [97]:
for learner in learner_name:
    print '%s: %d' % (learner, var[learner].value())

cnn: 0
gbdt: 1


In [67]:
for i in six.moves.range(20):
    cnn = pred_[i] + 0.0000001
    gbdt = pred_proba[i][1]
    model_sum = var[1] * math.log(cnn) + var[2] * math.log(gbdt)
    problem += model_sum
problem += var[1] >= 0
problem += var[2] >= 0
problem += var[1] + var[2] == 1

In [68]:
problem

localbayes:
MAXIMIZE
-16.1176042533*w_1 + -0.699505407252*w_2 + -0.0
SUBJECT TO
_C1: w_1 >= 0

_C2: w_2 >= 0

_C3: w_1 + w_2 = 1

VARIABLES
w_1 <= 1 Continuous
w_2 <= 1 Continuous

In [69]:
status = problem.solve()

In [71]:
pulp.LpStatus[status]

'Optimal'

In [72]:
var[1].value()

0.0

In [73]:
var[2].value()

1.0

### generate prediction of nn train point

In [127]:
n = 20
data_dim = len(nn_ts_ls[0])
nn_ts_array = numpy.array(nn_ts_ls).reshape((n, 1, 1, data_dim))
pred_cnn = []

for i in six.moves.range(n):
    x = np.asarray(nn_ts_array[i]).reshape((1, 1, 1, data_dim)).astype(numpy.float32)
    
    pred = F.softmax(model.predictor(chainer.Variable(x)))
    pred_cnn.append(pred.data[0][1])
    
pred_cnn = numpy.array(pred_cnn)
pred_gbdt = bdt.predict_proba(nn_ts_array.reshape(20, 30))[:, 1].astype(float32)

In [128]:
pred_cnn

array([  9.99988079e-01,   9.99928474e-01,   9.98474538e-01,
         6.80340047e-04,   9.99838948e-01,   9.99916911e-01,
         9.98352885e-01,   9.99886036e-01,   9.99996543e-01,
         9.99998808e-01,   1.77823881e-06,   8.33334343e-04,
         9.99953866e-01,   6.94811490e-07,   1.60206982e-03,
         9.99981165e-01,   9.78350818e-01,   2.32334484e-34,
         2.76201645e-05,   9.99780357e-01], dtype=float32)

In [129]:
pred_gbdt

array([ 0.50223386,  0.49973291,  0.50187421,  0.49409854,  0.50157022,
        0.50104266,  0.50086457,  0.5019697 ,  0.50106883,  0.502235  ,
        0.49648082,  0.49746409,  0.50287098,  0.49303561,  0.49671853,
        0.5024246 ,  0.50144929,  0.48345795,  0.49292389,  0.50001854], dtype=float32)

In [ ]:
learner_name = ['cnn', 'gbdt']

In [ ]:
problem = pulp.LpProblem('localbayes', pulp.LpMaximize)
var = pulp.LpVariable.dicts('w', (learner_name_ls), 0, 1, 'Continuous')

In [ ]:
for i in six.moves.range(20):
    cnn = pred_[i] + 0.0000001
    gbdt = pred_proba[i][1]
    model_sum = var[1] * math.log(cnn) + var[2] * math.log(gbdt)
    problem += model_sum

In [ ]:
for learner in learner_name:
    problem += var[learner] >= 0

In [ ]:
for learner in learner_name:
    problem += var[learner]
problem == 1